In [1]:
%matplotlib inline
import torch.optim as optim
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0,
                    8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [3]:
def model(t_u, w2, w1, b):
    return w2*t_u**2 + w1*t_u + b

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [4]:
def training_loop(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [5]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate) # <1>

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params, # <1> 
    t_u = t_un,
    t_c = t_c)

Epoch 500, Loss 10.708596
Epoch 1000, Loss 8.642083
Epoch 1500, Loss 7.171005
Epoch 2000, Loss 6.123478
Epoch 2500, Loss 5.377227
Epoch 3000, Loss 4.845286
Epoch 3500, Loss 4.465788
Epoch 4000, Loss 4.194724
Epoch 4500, Loss 4.000802
Epoch 5000, Loss 3.861744


tensor([ 0.5570, -0.8881, -0.8753], requires_grad=True)

In [6]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate) # <1>

training_loop(
    n_epochs = 2000, 
    optimizer = optimizer,
    params = params,
    t_u = t_u, # <2> 
    t_c = t_c)

Epoch 500, Loss 4.844499
Epoch 1000, Loss 3.785645
Epoch 1500, Loss 3.466678
Epoch 2000, Loss 3.406404


tensor([ 0.0060, -0.1081, -1.4510], requires_grad=True)

# Validation

In [7]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

torch.manual_seed(42)
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [8]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)

        with torch.no_grad(): # <1>
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad == False # <2>
            
        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validatiobn loss {val_loss.item():.4f}")        
            
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
    
    return params

In [9]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)

learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un, # <1> 
    val_t_u = val_t_un, # <1> 
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 700.8784, Validatiobn loss 562.9166
Epoch 2, Training loss 400.5066, Validatiobn loss 339.8012
Epoch 3, Training loss 231.7319, Validatiobn loss 209.3004
Epoch 500, Training loss 11.9179, Validatiobn loss 4.7955
Epoch 1000, Training loss 9.5623, Validatiobn loss 3.3341
Epoch 1500, Training loss 7.9316, Validatiobn loss 2.3322
Epoch 2000, Training loss 6.8021, Validatiobn loss 1.6464
Epoch 2500, Training loss 6.0192, Validatiobn loss 1.1780
Epoch 3000, Training loss 5.4761, Validatiobn loss 0.8588


tensor([ 0.5074, -0.5697, -0.7250], requires_grad=True)

In [10]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)

learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un, # <1> 
    val_t_u = val_t_un, # <1> 
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 700.8784, Validatiobn loss 562.9166
Epoch 2, Training loss 501.3456, Validatiobn loss 411.8047
Epoch 3, Training loss 336.9006, Validatiobn loss 285.1897
Epoch 500, Training loss 3.2683, Validatiobn loss 0.3116
Epoch 1000, Training loss 2.8387, Validatiobn loss 0.4661
Epoch 1500, Training loss 2.5311, Validatiobn loss 0.7121
Epoch 2000, Training loss 2.3825, Validatiobn loss 0.9766
Epoch 2500, Training loss 2.3346, Validatiobn loss 1.1795
Epoch 3000, Training loss 2.3250, Validatiobn loss 1.2913


tensor([  0.2500,   2.8272, -11.3123], requires_grad=True)